# Generative poems

## Prompting for completions

In [1]:
import sys
sys.path.append('../')
from generative_formalism import *

In [2]:
def clean_poem_str(txt):
    stanzas = txt.split('\n\n')
    stanzas = [st.strip() for st in stanzas if st.strip().count('\n')>0]
    return '\n\n'.join(stanzas)

def get_num_lines(txt):
    return len([x for x in txt.split('\n') if x.strip()])

### From previous work

In [3]:
def collect_prev_genai_completions(overwrite=False):
    index='_id	_first_n_lines	_model	_say_poem	_version	_timestamp'.split()
    df3 = pd.read_pickle('/Users/ryan/Dropbox/Prof/Data/data.output.gen_poems.v3.pkl').assign(_say_poem=True).reset_index().set_index(index)
    df4=pd.read_pickle('/Users/ryan/Dropbox/Prof/Data/data.output.gen_poems.v4.pkl').assign(_say_poem=True).reset_index().set_index(index)
    df5=pd.read_pickle('/Users/ryan/Dropbox/Prof/Data/data.output.gen_poems.v5.pkl').assign(_say_poem=True).reset_index().set_index(index)
    df6=pd.read_pickle('/Users/ryan/Dropbox/Prof/Data/data.output.gen_poems.v6.pkl')
    df7=pd.read_pickle('/Users/ryan/Dropbox/Prof/Data/data.output.gen_poems.v7.pkl')

    df = pd.concat([
        df3,#.query('_model=="ollama/llama3.1:8b"'),
        df4,#.query('_model=="ollama/llama3.1:8b-text-q4_K_M" | _model=="ollama/mistral" | _model=="ollama/mistral:text"'),
        df5,#.query('_model=="ollama/llama3.1:8b"'),
        df6,
        df7
    ]).reset_index()
    # df = df[df._model.str.contains('ollama/llama')]
    df = df[df._say_poem]
    df = df[~df._model.str.contains('poetry')]
    df = df.drop_duplicates(['_model','_id', 'stanza_num', 'line_num'])
    return df

In [4]:
df = collect_prev_genai_completions()
df

,_id,_first_n_lines,_model,_say_poem,_version,_timestamp,stanza_num,line_num,line_real,line_gen
0,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,1,Silent as a falling leaf,
1,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,2,To my heart there came a grief:,
2,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,3,"With a cold and pure despair,",
3,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,4,"Angerless, it settled there:",
4,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,5,"And must linger, and must stay,",
...,...,...,...,...,...,...,...,...,...,...
626298,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,4,16,'With anguish in your soul.',Thy life hath proved a whole.
626299,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,5,17,"Consider, you that slight the Lord,","Though priests were mine, and solemn days"
626300,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,5,18,Before his wrath appear;,"Observed with outward air,"
626301,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,5,19,"If once you fall beneath his sword,","Thy foot hath trod my sacred ways,"


### Replicating

In [5]:
# todo

### Postprocessing

In [8]:
# Filter out recognized completions
def filter_recognized_completions(df):
    # Test for line similarity
    df['line_sim'] = df.progress_apply(
        lambda row: fuzz.ratio(
            row.line_real.strip(), 
            row.line_gen.strip()
        ) if row.line_gen and row.line_real else np.nan, 
        axis=1
    )
    return df
    
    gby=[x for x in df if x[0]=='_']
    num1=len(df.groupby(gby))
    grps=[]
    grps_unsafe=[]
    for g,gdf in df.groupby(gby): 
        if gdf.line_sim.max()<95:
            grps.append(gdf)
        else:
            grps_unsafe.append(gdf)
    gdf = random.choice(grps)
    df_safe = pd.concat(grps)
    df_unsafe = pd.concat(grps_unsafe)
    print(f'Filtered out {num1 - len(grps)} recognized poems')
    return df_safe

In [9]:

df_safe = filter_recognized_completions(df)
df_safe

  0%|          | 0/326862 [00:00<?, ?it/s]

100%|██████████| 326862/326862 [00:03<00:00, 98738.96it/s] 


,_id,_first_n_lines,_model,_say_poem,_version,_timestamp,stanza_num,line_num,line_real,line_gen,line_sim
0,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,1,Silent as a falling leaf,,NaN
1,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,2,To my heart there came a grief:,,NaN
2,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,3,"With a cold and pure despair,",,NaN
3,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,4,"Angerless, it settled there:",,NaN
4,modern/ent2601/Z300573872,5,deepseek/deepseek-chat,True,1,1.742137e+09,1,5,"And must linger, and must stay,",,NaN
...,...,...,...,...,...,...,...,...,...,...,...
626298,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,4,16,'With anguish in your soul.',Thy life hath proved a whole.,35.087719
626299,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,5,17,"Consider, you that slight the Lord,","Though priests were mine, and solemn days",34.210526
626300,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,5,18,Before his wrath appear;,"Observed with outward air,",44.000000
626301,english/wattsisa/Z300522439,5,claude-3-sonnet-20240229,True,1,1.742790e+09,5,19,"If once you fall beneath his sword,","Thy foot hath trod my sacred ways,",37.681159


In [10]:
# To poem format
def get_num_lines(txt):
    return len([x for x in txt.split('\n') if x.strip()])

def to_poem_format(df, min_num_lines=10):
    gby=[x for x in df if x[0]=='_']
    ld = []
    done=set()
    for g,gdf in df.groupby(gby):
        first_n_lines = gdf['_first_n_lines'].iloc[0]
        poem_real = gdf.line_real[first_n_lines:].str.cat(sep='\n').strip()
        poem_gen = gdf.line_gen[first_n_lines:].str.cat(sep='\n').strip()
        
        d_real = {
            'id':gdf['_id'].iloc[0],
            'id_human':gdf['_id'].iloc[0],
            'model':'',
            'txt':poem_real,
            'num_lines':get_num_lines(poem_real),
            'line_sim':np.nan,
        }
        d_gen = {
            'id':get_id_hash_str(poem_gen),
            'id_human':gdf['_id'].iloc[0],
            'model':gdf['_model'].iloc[0],
            'txt':poem_gen,
            'num_lines':get_num_lines(poem_gen),
            'line_sim':gdf['line_sim'].max(),
        }

        done_id = (d_gen['id'], d_gen['model'])
        if d_gen['num_lines'] >= min_num_lines and d_real['num_lines'] >= min_num_lines and done_id not in done:
            done.add(done_id)
            ld.append(d_real)
            ld.append(d_gen)
    
    odf = pd.DataFrame(ld)
    return odf

In [11]:
df_poems = to_poem_format(df_safe)
df_poems

,id,id_human,model,txt,num_lines,line_sim
0,african-american/benjamin/Z200277076,african-american/benjamin/Z200277076,,And no longer submit to tyrant's rule;\n'Tis y...,15,NaN
1,193cc9b5,african-american/benjamin/Z200277076,ollama/olmo2:latest,"Embrace equality, let no chains confound,\n F...",12,43.678161
2,african-american/benjamin/Z200277079,african-american/benjamin/Z200277079,,None so beauteous or true;\nHer equal I could ...,11,NaN
3,10dee82d,african-american/benjamin/Z200277079,ollama/olmo2:latest,"From her lips drink honeyed words with glee,\n...",11,50.666667
4,african-american/benjamin/Z200277084,african-american/benjamin/Z200277084,,Were dark as the raven's black wing;\nAnd I th...,15,NaN
...,...,...,...,...,...,...
22453,4b0ba84a,modern/sci0101/Z200480982,ollama/llama3.1:8b-text-q4_K_M,"""photographic chambers,""\nthough they were sti...",12,45.783133
22454,modern/sci0101/Z200480982,modern/sci0101/Z200480982,,"""mousetraps"".\nFox Talbot researched the etymo...",12,NaN
22455,ee89973f,modern/sci0101/Z200480982,ollama/mistral:text,"the ""photographing plates,"" and the pictures\n...",12,44.444444
22456,modern/sci0101/Z200480982,modern/sci0101/Z200480982,,"""mousetraps"".\nFox Talbot researched the etymo...",12,NaN


In [12]:
def save_poem_format_completions(df, overwrite=False):
    ofn='../data/corpus_genai_completions.csv.gz'
    save_sample(df.set_index('id'), ofn, overwrite=overwrite)


In [13]:
save_poem_format_completions(df_poems)

Saved sample to ../data/corpus_genai_completions.csv.gz
